In [1]:
#r "./bin/Debug/net5.0/PreStormCore.dll"
#r "./bin/Debug/net5.0/Demo.dll"
using System.Diagnostics;
using System.Xml.Linq;
using PreStormCore;

var sfcrimes = new Sfcrimes.Service();
var usa = new Usa.Service();

var crimes = sfcrimes.CrimeLocations.Download() //extraParameters: "outSR=4326", keepQuerying: true, degreeOfParallelism: 8)
    .ToArray();

var cities = usa.Cities.Download(null, null, true, 4).ToArray();
var highways = usa.Highways.Download().ToArray();
var states = usa.States.Download().ToArray();

static void Show(this IEnumerable<dynamic> features) {
    features.OfType<Feature>().ToKml(placemarkElements: x => new[]{ new XElement("Style",
        new XElement("IconStyle", new XElement("color", "ffffffff"), new XElement("Icon", new XElement("href", "http://maps.google.com/mapfiles/kml/shapes/square.png"))),
        new XElement("PolyStyle", new XElement("color", "b3ffffff"))
        )})
        .Save("doc.kml");
    Process.Start(new ProcessStartInfo("doc.kml"){ UseShellExecute = true });
}


In [1]:
#r "nuget: XPlot.Plotly, 3.0.1"
XPlot.Plotly.Chart.Column(crimes.GroupBy(x => x.Pddistrict).OrderByDescending(x => x.Count()).Select(x => Tuple.Create(x.Key, x.Count())))


In [1]:
#r "nuget: XPlot.Plotly, 3.0.1"
XPlot.Plotly.Chart.Pie(crimes.GroupBy(x => x.Category).OrderByDescending(x => x.Count()).Select(x => Tuple.Create(x.Key, x.Count())))


In [1]:
cities.Where(x => x.St == "OH").Show()


In [1]:
highways.Show()


In [1]:
states.Where(x => x.StateAbbr == "AK").Show()


In [1]:
using RedlandsEmergencyVehicles;

var redlands = new RedlandsEmergencyVehicles.Service();
var ambulance = new Ambulance {
    Unitname = "Dummy",
    Status = UnitStatus.Active,
    Speed = 40,
    Lastreport = DateTime.UtcNow,    
    Geometry = (-80, 45)
};

var result = await redlands.Ambulances.InsertAsync(ambulance);
redlands.Ambulances.Download().Show();
result.InsertedFeatures


index,Unitname,Status,Type,Callnumber,Speed,Lastreport,Geometry,OID,IsDataBound,IsDirty
0,Dummy,Active,<null>,<null>,40,2021-02-13 22:50:38Z,PreStormCore.Point,391440,True,False


In [1]:
using RedlandsEmergencyVehicles;

var redlands = new RedlandsEmergencyVehicles.Service();
var ambulances = redlands.Ambulances.Download("Unitname='Dummy'").ToArray();

var result = await redlands.Ambulances.DeleteAsync(ambulances);
redlands.Ambulances.Download().Show();
result.Raw.deleteResults


index,objectId,success
0,391831,True
1,391439,True


In [1]:
using Wildfire;

var wildfire = new Wildfire.Service();

await wildfire.WildfireResponsePolygons.InsertAsync(new WildfireResponsePolygon{
    Description = "Delete me!",
    Geometry = new Polygon((-121, 50), (-121, 51), (-120, 51), (-120, 50))
});

var fires = wildfire.WildfireResponsePolygons.Download("description='Delete me!'").ToArray();
fires.Show();

var result = await wildfire.WildfireResponsePolygons.DeleteAsync(fires);
result.Raw.deleteResults


index,objectId,success
0,2801512,True
